In [1]:
import os
import numpy as np  
import pandas as pd

In [24]:
jokes = pd.read_csv("dataset_joke/JokeText.csv")
data = pd.read_csv("dataset_joke/UserRatings1.csv", index_col="JokeId")
data.head()

,User1,User2,User3,User4,User5,User6,User7,User8,User9,User10,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
JokeId,,,,,,,,,,,,,,,,,,,,,
0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,-7.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.91,NaN
1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,-5.15,...,NaN,NaN,NaN,-5.63,NaN,-6.07,NaN,-1.60,-4.56,NaN
2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,-3.25,...,NaN,NaN,NaN,NaN,NaN,4.08,NaN,NaN,8.98,NaN
3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,-1.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,4.03,...,2.28,-0.49,5.1,-0.29,-3.54,-1.36,7.48,-5.78,0.73,2.62


In [25]:
data = data.iloc[:, :5000]
sums_of_col = data.sum(axis=1)
col_size = len(data.columns)
value = sums_of_col / col_size
index_of_max = value[value == value.max()].index[0]


In [61]:
# ユーザベクトル間のコサイン距離を求める
# cosine_similarity/2 だけでなく、cosine_similarity/1 も用いることができる
# `If Y is omitted the pairwise distances of the row vectors of X are calculated`
# https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity

from sklearn.metrics.pairwise import cosine_similarity
data = data.T
filetering_cosim = cosine_similarity(data)
filetering_cosim.shape



(5000, 5000)

In [62]:
# 0番ユーザに最も近い10人を選び出す

most_sim_users = sorted(list(enumerate(filetering_cosim[0])), key=lambda x: x[1], reverse=True)
most_sim_users = most_sim_users[1:11]
sim_users = [x[0] for x in most_sim_users]
sim_users


[4128, 4946, 3734, 1825, 2304, 2189, 474, 870, 543, 2137]

In [63]:
candidate_jokes = data.iloc[sim_users, :]
candidate_jokes.head


<bound method NDFrame.head of JokeId      0     1     2     3     4     5     6     7     8     9   ...  \
User4129  3.98  7.82  0.78 -1.60  1.99  1.17 -8.79  2.14  0.78  2.38  ...   
User4947  4.32  0.19  2.77 -0.97 -0.39  2.91 -2.48 -0.29  0.19  1.84  ...   
User3735 -0.24  0.15  2.14 -3.11 -3.69  3.20  1.12  0.39  0.34 -0.05  ...   
User1826  3.25  4.03  3.54 -1.26  3.16  2.96  1.70 -6.99  6.50  4.22  ...   
User2305  8.16  3.40  7.23 -5.39  4.03  6.65  7.23  6.84 -3.25 -5.39  ...   
User2190  9.22  3.69  9.08 -0.97  3.74  8.93  3.40  2.33  9.22  6.55  ...   
User475   8.88 -9.90  9.27  3.79 -8.64  9.03 -9.42  3.88  4.51  0.68  ...   
User871   6.46  1.84  8.88  0.19  3.11  2.04  7.67  7.18  5.24  8.20  ...   
User544   9.32  0.58  7.86  1.94  9.22 -0.05  4.03  2.28  3.20  4.03  ...   
User2138  4.71  9.22  9.22  5.87  8.88 -1.26 -9.37 -4.37  4.32  7.82  ...   

JokeId      90    91    92    93    94    95    96    97    98    99  
User4129  6.02 -0.15 -9.56  1.07 -7.33 -1.99 -9.66 

In [66]:
## ジョークの出来の平均点を求めて、高い順に勧めれば良さそう
mean_score = candidate_jokes.mean(axis=0)
mean_score = mean_score.sort_values(axis=0, ascending=False)
mean_score

JokeId
71    6.776
61    6.741
55    6.736
76    6.709
34    6.664
      ...  
36   -1.728
15   -4.679
12   -5.461
57   -5.994
14   -7.077
Length: 100, dtype: float64

In [ ]:
## 一般化すると：
def UBCF(user_num):
    ## 近い嗜好の人上位10人を探す
    most_sim_users =  sorted(list(enumerate(filetering_cosim[user_num])), key=lambda x: x[1], reverse=True)
    most_sim_users = most_sim_users[1:11]

    ## 各々のユーザidと、値を取り出す
    sim_users = [x[0] for x in most_sim_users]
    sim_value = [x[1] for x in most_sim_users]

    ## ジョークごとの平均点を求める 
    candidate_jokes = data.iloc[sim_users, :]
    mean_score = pd.Series(candidate_jokes.mean(axis=0))
    mean_score = mean_score.sort_values(axis=0, ascending=False)

    ## 平均点が高い順に勧める
    recom_jokes = list(mean_score = mean_score.sort_values(axis=0, ascending=False))
        for i in recom_jokes:
        print("Index Number {} and Joke is {} :".format(i,jokes.iloc[i,:]))
    return(recom_jokes)
    return
